In [1]:
library("tidyverse")
library("phyloseq")
library('qiime2R')
library("ggpubr")
library("MASS")
library("scales")
library("caret")
library("AppliedPredictiveModeling")
library("ranger")
library("e1071")
library("randomForest")
library("alluvial")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract


Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select


Attaching package: ‘scales’

The following object is masked from ‘package:purrr’:

    discard

The following object is masked from ‘package:readr’:

    col_factor

Loading required package: lattice

Attaching package: ‘caret’

The following object is masked from ‘package:purrr’:

    lift

randomForest 4.6-14
Type r

### Setup, Functions, and loading data


In [2]:
# Set seed for analysis
set.seed(10031993)
# Theme, color palette, and working directory
theme_set(theme_pubr())
zoe_palette <- c("gray","#1b9e77", "#7570b3",  "#e6ab02")

# Function to return metadata df from phyloseq object
pssd2veg <- function(physeq) {
  # From a phyloseq object return a dataframe of the sample metadata for use in vegan
  # From: https://jacobrprice.github.io/2017/08/26/phyloseq-to-vegan-and-back.html
  sd <- sample_data(physeq)
  return(as(sd,"data.frame"))
}
# Function to plot or run a linear model on a ASV from a phyloseq object
plot_deseq2_DiffAbunMicob <- function(physeq_obj, ASV_number){
  temp_sample_tab <- pssd2veg(physeq_obj)
  otu_matrix <- as(otu_table(physeq_obj), "matrix")
  TEMP <- data.frame(ASV_count = otu_matrix[ASV_number,])
  TEMP2<- cbind(temp_sample_tab, TEMP)
  #Anova(lm(ASV_count ~ Tissue*Rootstock + Tissue*Irrigation + Rootstock*Irrigation + Block, data = TEMP2), type = "III")
  ggplot(TEMP2, aes(Tissue, ASV_count, fill= Rootstock)) + geom_boxplot(outlier.shape = NA) + geom_point(position = position_jitterdodge(jitter.width = 0.2)) + scale_fill_manual(name = "Rootstock", values=zoe_palette) + scale_y_continuous(name="Abundance") + xlab("Tissue") + theme(legend.position="right", axis.title = element_text(size = 14), axis.text = element_text(size = 12), plot.title = element_text(size=22))
}

# Function to plot confusion matrix using ggtile plot from a confussion matrix object
# By user: Enrique P?rez Herrero 
# on https://stackoverflow.com/questions/46063234/how-to-produce-a-confusion-matrix-and-find-the-misclassification-rate-of-the-na%C3%AF
ggplotConfusionMatrix <- function(m){
  mytitle <- paste("Accuracy", percent_format()(m$overall[1]),
                   "Kappa", percent_format()(m$overall[2]))
  
  d <- as.data.frame.matrix(m$table)
  drn <- colnames(d)
  drr <- rownames(d)
  drs <- rowSums(d)
  d <- d %>% mutate_if(is.numeric, funs(./drs))
  d <- d %>% gather(x, value)
  Y <- cbind(as.data.frame(m$table), Proportion = d$value)

  p <-
    ggplot(data = Y, aes(x = Reference, y = Prediction, fill= Proportion)) +
    geom_tile( colour = "white") +
    scale_fill_gradient(low = "white", high = "#14A02E", na.value = "white", limits=c(0,1)) +
    ggtitle(mytitle) +
    # ADDED LINE HERE FOR ANGLE OF X AXIS LABELS
    theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
    theme(legend.position = "right") +
    guides(fill = guide_colorbar(frame.colour = "black", ticks = FALSE))
  return(p)
}

MachineLearning_RF_ranger <- function(PHYSEQ_OBJ_1, PHYSEQ_OBJ_2 = FALSE, GROUPING, NUM_TREES = 101) {
  if (missing(PHYSEQ_OBJ_2)){
    # Remove ASV Table and meta data from phyloseq objects
    # 16s
    ASV.df <- as.data.frame(otu_table(PHYSEQ_OBJ_1))
    ASV_metadata.df <- as.data.frame(sample_data(PHYSEQ_OBJ_1))
    # Format ASV table to be used for machine learning applications and make metadata df
    ASV.df <- t(ASV.df)
    ASV_meta.df <- data.frame(Sample = rownames(ASV_metadata.df), Rootstock = ASV_metadata.df$Rootstock, Tissue = ASV_metadata.df$Tissue, Tissue_Rootstock = paste(ASV_metadata.df$Tissue, ASV_metadata.df$Rootstock, sep = "_"))
    ASV_prefiltered.df <- cbind(ASV.df, ASV_meta.df)
    # ~80:20 split train/test datasets while respecting groups (i.e. sampling the same number of samples from each label)
    # 47 berry, leaf, root, and soil samples
    if ((GROUPING) == "Tissue_Rootstock"){
      train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(9))
    } else {
      train_index <- as.data.frame(ASV_prefiltered.df %>% group_by_(GROUPING) %>% sample_n(36))
    }
    rownames(train_index) <- train_index$Sample
    train_index <- match(rownames(train_index), rownames(ASV_prefiltered.df))
    train_x <- as.data.frame(ASV.df[train_index, ])
    test_y <- as.data.frame(ASV.df[-train_index, ])
    # Train set, 144 samples
    train_x$Sample <- rownames(train_x)
    Training_meta.df <- merge(train_x, ASV_meta.df, by = 'Sample')
    train_x <- subset(Training_meta.df, select = -c(Rootstock, Tissue, Tissue_Rootstock))
    rownames(train_x) <- train_x$Sample
    train_x <- subset(train_x, select = -c(Sample))
    Training_meta.df <- subset(Training_meta.df, select = c(Sample, Rootstock, Tissue, Tissue_Rootstock))
    rownames(Training_meta.df) <- Training_meta.df$Sample 
    # Test set, 40 samples
    test_y$Sample <- rownames(test_y)
    Testing_meta.df <- merge(test_y, ASV_meta.df, by = "Sample")
    test_y <- subset(Testing_meta.df, select = -c(Rootstock, Tissue, Tissue_Rootstock))
    rownames(test_y) <- test_y$Sample
    test_y <- subset(test_y, select = -c(Sample))
    Testing_meta.df <- subset(Testing_meta.df, select = c(Sample, Rootstock, Tissue, Tissue_Rootstock))
    rownames(Testing_meta.df) <- Testing_meta.df$Sample 
    # Training model
    Training_grid <- expand.grid(.mtry = seq(10, length(train_x), round(length(train_x)*0.1)), .splitrule= "gini", .min.node.size = c(1, 5, 10))
    RF_CM <- list()
    RF_CM[["RF_model"]] <- train(x = train_x, y = Training_meta.df[[GROUPING]], method = "ranger", importance = "impurity", tuneGrid = Training_grid, num.trees = NUM_TREES)
    RF_prediction_3 <- predict(RF_CM[["RF_model"]], test_y)
    RF_CM[["CMatrix"]] <- confusionMatrix(RF_prediction_3, as.factor(Testing_meta.df[[GROUPING]]))
    RF_CM[["CMatrixPLOT"]] <- ggplotConfusionMatrix(RF_CM[["CMatrix"]])
    RF_CM[["VarImporance"]] <- varImp(RF_CM[["RF_model"]], scale = FALSE)
    RF_CM
  } else {
    # Remove ASV Table and meta data from phyloseq objects
    # 16s
    ASV_16s.df <- as.data.frame(otu_table(PHYSEQ_OBJ_1))
    ASV_16s_metadata.df <- as.data.frame(sample_data(PHYSEQ_OBJ_1))
    # its
    ASV_its.df <- as.data.frame(otu_table(PHYSEQ_OBJ_2))
    ASV_its_metadata.df <- as.data.frame(sample_data(PHYSEQ_OBJ_2))
    # Combine 16s and ITS data into a single matrix and remove samples that are not present both 16s and ITS datasets
    setdiff(rownames(t(ASV_16s.df)), rownames(t(ASV_its.df)))
    ASV_16s_4removed.df <- subset(ASV_16s.df, select = -c(`MV11.3309C.2-3.S`, `MV14.3309C.2-3.R`, `MV15.1103P.10-11.R`, `MV16.3309C.14-15.S`))
    # merge data frames
    Merged_16s_its.df <- rbind(ASV_16s_4removed.df, ASV_its.df)
    # Format ASV table to be used for machine learning applications and make metadata df
    ASV_both.df <- t(Merged_16s_its.df)
    ASV_meta.df <- data.frame(Sample = rownames(ASV_its_metadata.df), Rootstock = ASV_its_metadata.df$Rootstock, Tissue = ASV_its_metadata.df$Tissue, Tissue_Rootstock = paste(ASV_its_metadata.df$Tissue, ASV_its_metadata.df$Rootstock, sep = "_"))
    ASV_both_w_meta.df <- cbind(ASV_both.df, ASV_meta.df)
    # ~80:20 split train/test datasets while respecting groups (i.e. sampling the same number of samples from each label)
    # 47 berry, leaf, root, and soil samples
    if ((GROUPING) == "Tissue_Rootstock"){
      train_index <- as.data.frame(ASV_both_w_meta.df %>% group_by_(GROUPING) %>% sample_n(9))
    } else {
      train_index <- as.data.frame(ASV_both_w_meta.df %>% group_by_(GROUPING) %>% sample_n(36))
    }
    rownames(train_index) <- train_index$Sample
    train_index <- match(rownames(train_index), rownames(ASV_both_w_meta.df))
    train_x <- as.data.frame(ASV_both.df[train_index, ])
    test_y <- as.data.frame(ASV_both.df[-train_index, ])
    # Train set, 144 samples
    train_x$Sample <- rownames(train_x)
    Training_meta.df <- merge(train_x, ASV_meta.df, by = 'Sample')
    train_x <- subset(Training_meta.df, select = -c(Rootstock, Tissue, Tissue_Rootstock))
    rownames(train_x) <- train_x$Sample
    train_x <- subset(train_x, select = -c(Sample))
    Training_meta.df <- subset(Training_meta.df, select = c(Sample, Rootstock, Tissue, Tissue_Rootstock))
    rownames(Training_meta.df) <- Training_meta.df$Sample 
    # Test set, 40 samples
    test_y$Sample <- rownames(test_y)
    Testing_meta.df <- merge(test_y, ASV_meta.df, by = "Sample")
    test_y <- subset(Testing_meta.df, select = -c(Rootstock, Tissue, Tissue_Rootstock))
    rownames(test_y) <- test_y$Sample
    test_y <- subset(test_y, select = -c(Sample))
    Testing_meta.df <- subset(Testing_meta.df, select = c(Sample, Rootstock, Tissue, Tissue_Rootstock))
    rownames(Testing_meta.df) <- Testing_meta.df$Sample 
    #Train model predict, and create confusion matrix
    Training_grid <- expand.grid(.mtry = seq(10, length(train_x), round(length(train_x)*0.1)), .splitrule= "gini", .min.node.size = c(1, 5, 10))
    RF_CM <- list()
    RF_CM[["RF_model"]] <- train(x = train_x, y = Training_meta.df[[GROUPING]], method = "ranger", importance = "impurity", tuneGrid = Training_grid, num.trees = NUM_TREES)
    RF_prediction_3 <- predict(RF_CM[["RF_model"]], test_y)
    RF_CM[["CMatrix"]] <- confusionMatrix(RF_prediction_3, as.factor(Testing_meta.df[[GROUPING]]))
    RF_CM[["CMatrixPLOT"]] <- ggplotConfusionMatrix(RF_CM[["CMatrix"]])
    RF_CM[["VarImporance"]] <- varImp(RF_CM[["RF_model"]], scale = FALSE)
    RF_CM
  }
}

In [3]:
physeq_16s <- readRDS("physeq_16s.Rds")
physeq_its <- readRDS("physeq_its.Rds")

In [4]:
# Recode factors OWN to Ungrafted
physeq_16s@sam_data$Rootstock <- recode_factor(physeq_16s@sam_data$Rootstock , OWN = "Ungrafted")
physeq_its@sam_data$Rootstock <- recode_factor(physeq_its@sam_data$Rootstock , OWN = "Ungrafted")

In [5]:
##### 1.0 | 16s and ITS data together, 4 samples removed from 16s, and no preprocessing or prefiltering of ASVs. #####
RF_1.1 <- MachineLearning_RF_ranger(physeq_16s, physeq_its, GROUPING = "Rootstock", NUM_TREES = 314)
RF_1.2 <- MachineLearning_RF_ranger(physeq_16s, physeq_its, GROUPING = "Tissue", NUM_TREES = 381)
RF_1.3 <- MachineLearning_RF_ranger(physeq_16s, physeq_its, GROUPING = "Tissue_Rootstock", NUM_TREES = 324)

Warning message:
“group_by_() is deprecated. 
Please use group_by() instead

The 'programming' vignette or the tidyeval book can help you
to program with group_by() : https://tidyeval.tidyverse.org
This warning is displayed once per session.”Warning message:
“funs() is soft deprecated as of dplyr 0.8.0
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once per session.”

In [12]:
RF_1.1$RF_model

Random Forest 

 144 samples
9394 predictors
   4 classes: 'Ungrafted', '1103P', '3309C', 'SO4' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 144, 144, 144, 144, 144, 144, ... 
Resampling results across tuning parameters:

  mtry  min.node.size  Accuracy   Kappa       
    10   1             0.2179348  -0.014970692
    10   5             0.2174720  -0.015595671
    10  10             0.2305815   0.001507861
   949   1             0.3672886   0.168895941
   949   5             0.3578489   0.155622901
   949  10             0.3678063   0.169096536
  1888   1             0.3799966   0.184979892
  1888   5             0.3702900   0.171596647
  1888  10             0.3792755   0.183249920
  2827   1             0.3787848   0.182121606
  2827   5             0.3885524   0.195138466
  2827  10             0.3674067   0.167749875
  3766   1             0.3750189   0.177053604
  3766   5             0.3864066   0.191437046
  3766  10             0.3726334   0.

In [13]:
RF_1.2$RF_model

Random Forest 

 144 samples
9394 predictors
   4 classes: 'Berry', 'Leaf', 'Root', 'Soil' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 144, 144, 144, 144, 144, 144, ... 
Resampling results across tuning parameters:

  mtry  min.node.size  Accuracy   Kappa    
    10   1             0.8596497  0.8153643
    10   5             0.8636361  0.8206330
    10  10             0.8616578  0.8181316
   949   1             0.9831600  0.9774230
   949   5             0.9854089  0.9804295
   949  10             0.9824053  0.9764154
  1888   1             0.9800192  0.9732146
  1888   5             0.9817036  0.9754713
  1888  10             0.9817036  0.9754713
  2827   1             0.9815296  0.9752371
  2827   5             0.9815543  0.9752830
  2827  10             0.9808283  0.9742927
  3766   1             0.9784768  0.9711425
  3766   5             0.9785565  0.9712243
  3766  10             0.9770600  0.9692238
  4705   1             0.9738510  0.9649049

In [14]:
RF_1.3$RF_model

Random Forest 

 144 samples
9394 predictors
  16 classes: 'Berry_1103P', 'Berry_3309C', 'Berry_SO4', 'Berry_Ungrafted', 'Leaf_1103P', 'Leaf_3309C', 'Leaf_SO4', 'Leaf_Ungrafted', 'Root_1103P', 'Root_3309C', 'Root_SO4', 'Root_Ungrafted', 'Soil_1103P', 'Soil_3309C', 'Soil_SO4', 'Soil_Ungrafted' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 144, 144, 144, 144, 144, 144, ... 
Resampling results across tuning parameters:

  mtry  min.node.size  Accuracy   Kappa    
    10   1             0.1609761  0.1259447
    10   5             0.1584087  0.1241194
    10  10             0.1580139  0.1233453
   949   1             0.2497978  0.2113840
   949   5             0.2428230  0.2035286
   949  10             0.2498001  0.2111543
  1888   1             0.2473579  0.2083031
  1888   5             0.2478270  0.2081808
  1888  10             0.2542999  0.2150218
  2827   1             0.2613154  0.2220747
  2827   5             0.2570167  0.2170257
  2827  10      

In [7]:
# Plotting and saving a object to combine in Rstudio

X <- ggarrange(RF_1.1[[3]], RF_1.2[[3]], RF_1.3[[3]], ncol = 3, common.legend = TRUE, legend = "right")
pdf("RF_model_for_plot.pdf", onefile = TRUE, width = 18, height = 12)
print(X)
dev.off()

png 
  2

### Testing different datasets across rootstock, tissue, and both.

In [8]:
# ##### 1.0 | 16s and ITS data together, 4 samples removed from 16s, and no preprocessing or prefiltering of ASVs. #####
# RF_1.1 <- MachineLearning_RF_ranger(physeq_16s, physeq_its, GROUPING = "Rootstock")
# RF_1.2 <- MachineLearning_RF_ranger(physeq_16s, physeq_its, GROUPING = "Tissue")
# RF_1.3 <- MachineLearning_RF_ranger(physeq_16s, physeq_its, GROUPING = "Tissue_Rootstock")

# ##### 2.0 | 16s and ITS data together with prefiltering of ASVs, 4 samples removed from 16s, and no preprocessing. #####
# # Filter dataset to eliminate ASVs that are mostly zeros
# # Remove taxa that are not found greater than 15 times in 10% of the samples
# # 16s: 8199 ASVs => 1084  | ITS: 1195 ASVs => 124
# physeq_16s_filtered <- filter_taxa(physeq_16s, function(x) sum(x > 15) > (0.10*length(x)), TRUE)
# physeq_its_filtered <- filter_taxa(physeq_its, function(x) sum(x > 15) > (0.10*length(x)), TRUE)
# RF_2.1 <- MachineLearning_RF_ranger(physeq_16s_filtered, physeq_its_filtered, GROUPING = "Rootstock")
# RF_2.2 <- MachineLearning_RF_ranger(physeq_16s_filtered, physeq_its_filtered, GROUPING = "Tissue")
# RF_2.3 <- MachineLearning_RF_ranger(physeq_16s_filtered, physeq_its_filtered, GROUPING = "Tissue_Rootstock")

# ##### 3.0 | 16s data, no prefiltering or preprocessing of ASVs. #####
# RF_3.1 <- MachineLearning_RF_ranger(physeq_16s, GROUPING = "Rootstock")
# RF_3.2 <- MachineLearning_RF_ranger(physeq_16s, GROUPING = "Tissue")
# RF_3.3 <- MachineLearning_RF_ranger(physeq_16s, GROUPING = "Tissue_Rootstock")

# ##### 4.0 | 16s data, no preprocessing of ASVs. #####
# # Filter dataset to eliminate ASVs that are mostly zeros
# # Remove taxa that are not found greater than 15 times in 10% of the samples
# # 16s: 8199 ASVs => 1084
# physeq_16s_filtered <- filter_taxa(physeq_16s, function(x) sum(x > 15) > (0.10*length(x)), TRUE)
# RF_4.1 <- MachineLearning_RF_ranger(physeq_16s_filtered, GROUPING = "Rootstock")
# RF_4.2 <- MachineLearning_RF_ranger(physeq_16s_filtered, GROUPING = "Tissue")
# RF_4.3 <- MachineLearning_RF_ranger(physeq_16s_filtered, GROUPING = "Tissue_Rootstock")

# ##### 5.0 | its data, no prefiltering or preprocessing of ASVs. #####
# RF_5.1 <- MachineLearning_RF_ranger(physeq_its, GROUPING = "Rootstock")
# RF_5.2 <- MachineLearning_RF_ranger(physeq_its, GROUPING = "Tissue")
# RF_5.3 <- MachineLearning_RF_ranger(physeq_its, GROUPING = "Tissue_Rootstock")

# ##### 6.0 | its data, no preprocessing of ASVs. #####
# # Filter dataset to eliminate ASVs that are mostly zeros
# # Remove taxa that are not found greater than 15 times in 10% of the samples
# # 16s: 8199 ASVs => 1084  | ITS: 1195 ASVs => 124
# physeq_its_filtered <- filter_taxa(physeq_its, function(x) sum(x > 15) > (0.10*length(x)), TRUE)
# RF_6.1 <- MachineLearning_RF_ranger(physeq_its_filtered, GROUPING = "Rootstock")
# RF_6.2 <- MachineLearning_RF_ranger(physeq_its_filtered, GROUPING = "Tissue")
# RF_6.3 <- MachineLearning_RF_ranger(physeq_its_filtered, GROUPING = "Tissue_Rootstock")

### Saving CM's of all datasets and models

In [9]:
# A <- annotate_figure(ggarrange(RF_1.1[[3]], RF_1.2[[3]], RF_1.3[[3]]), top = text_grob("16s and ITS data together, no filtering", color = "red", face = "bold", size = 14))
# B <- annotate_figure(ggarrange(RF_2.1[[3]], RF_2.2[[3]], RF_2.3[[3]]), top = text_grob("16s and ITS data together with prefiltering of ASVs", color = "red", face = "bold", size = 14))
# C <- annotate_figure(ggarrange(RF_3.1[[3]], RF_3.2[[3]], RF_3.3[[3]]), top = text_grob("16s data, no filtering", color = "red", face = "bold", size = 14))
# D <- annotate_figure(ggarrange(RF_4.1[[3]], RF_4.2[[3]], RF_4.3[[3]]), top = text_grob("16s data filtered", color = "red", face = "bold", size = 14))
# E <- annotate_figure(ggarrange(RF_5.1[[3]], RF_5.2[[3]], RF_5.3[[3]]), top = text_grob("ITS data, no filtering", color = "red", face = "bold", size = 14))
# F <- annotate_figure(ggarrange(RF_6.1[[3]], RF_6.2[[3]], RF_6.3[[3]]), top = text_grob("ITS data filtered", color = "red", face = "bold", size = 14))
# ggarrange(A, B, C, D, E, F)

In [10]:
# plot_list <- list(A, B, C, D, E, F)
# pdf("All_RF_models_and_datasets.pdf", onefile = TRUE, width = 18, height = 12)
# print(plot_list)
# dev.off()